
### Script Bronze to Silver producció
- Executa les comandes mínimes però necessaries per realitzar el procès més optim possible.
- Per veure com s'ha arribat a desenvolupar aquest df i que fa cada pas mirar el Notebook de documentació.

#### Importem les llibreries necessàries

In [0]:
import re
import numpy as np
import pandas as pd


#### Configuració connexió Spark

In [0]:
# Configuració Spark per connexió ADLG2 amb token SAS
token = 'sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-03T02:34:04Z&st=2024-04-09T18:34:04Z&spr=https&sig=dplYYYNSRC%2FjDR89WMxcsA6SMo%2BHFYrQ5BqdhRUKBMs%3D'
storage_account = 'projecteiabd'
container = 'bronze'
path = 'abfs://{0}@{1}.dfs.core.windows.net/'.format(container, storage_account)

spark.conf.set("fs.azure.account.auth.type.{0}.dfs.core.windows.net".format(storage_account), "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.{0}.dfs.core.windows.net".format(storage_account), "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.{0}.dfs.core.windows.net".format(storage_account), token)


#### Declaració de funcions.


In [0]:
# Funció per llistar tots els documents que hi ha dins de una ruta.

def deep_ls(path): # Definim una funció que li passem un path.
    """List all files in base path recursively."""
    for x in dbutils.fs.ls(path): # Agafem tots els fitxer i carpetas que hi ha dintre el path. I els anem recorrent. 
        if x.path[-1] != '/': # Agefm l'ultim caracter de la ruta del fitxer, mirem que si no és un '/' vol dir que és un fitxer.
            yield x # Retornem el fitxer amb un yield osigui que retorna el fitxer però segueix amb l'execuió fins que no quedin més docs.
        else: 
            for y in deep_ls(x.path): # Si no és un fitxer entrem dintre la carpeta, amb la mateixa funció de aqui bé la recursivitat. Que no parara de executar-se fins que ho hagi recorregut tot.
                yield y # Anem retornan els resultats de cada fitxer que va trobant la funció.

In [0]:
# Funció per eliminar columnes inecessaries.
def deleteColumns(df, columns_list):

  df_docs.drop(columns_list, inplace=True, axis = 1)

  return df_docs

In [0]:
# Funció per visualitzar els productes de un df.
def visualitzarProducts(df_products):
    for index, llista_productes in df_products['products'].items():
        print("Tiquet ", index + 1)
        print()
        print("--Productes--")
        for producte in llista_productes:
            print("Nom:", producte['descripcio'])
            print("Qty:", producte['quantitat'])
            print("Import:", producte['import'], "€")
            print('preu_unitari:', producte['preu_unitari'], '€')
            print('unitat:', producte['unitat'])
            print('')

In [0]:
# Funció per emmegatzemar en format parquet les dades a un repositori extern. Particionades per id_client.
def particionar_clients(df, path):
    # Passem el df de pandas a spark df.
    df = spark.createDataFrame(df)

    df.write.format("parquet") \
        .option("forward_spark_azure_storage_credentials", "True") \
        .partitionBy("id_client") \
        .mode("overwrite") \
        .save(path)

In [0]:
# Funció per emmegatzemar en format parquet les dades a un repositori extern. Particionades per dies.
def particionar_dies(df, path):
    # Passem el df de pandas a spark df.
    df = spark.createDataFrame(df)

    df.write.format("parquet") \
        .option("forward_spark_azure_storage_credentials", "True") \
        .partitionBy("data") \
        .mode("overwrite") \
        .save(path)

In [0]:
# Funció per eliminar el text "kg" de la columna preu unitats.
def eliminar_txt_kg(df):
    for index, llista_productes in df['products'].items():
        for producte in llista_productes:
            if 'preu_unitari' in producte:
                # Elimina el text " €/kg €"
                producte['preu_unitari'] = re.sub(r' ?€/kg ?', '', producte['preu_unitari'])
    return df


In [0]:
# Funció per canviar les , per . a l'import, preu_unitari i total_amb_IVA
def canviar_punts_per_comes(row):
    row['total_amb_IVA'] = str(row['total_amb_IVA']).replace(',', '.')
    for producte in row['products']:
        producte['import'] = str(producte['import']).replace(',', '.')
        if 'preu_unitari' in producte:
            producte['preu_unitari'] = str(producte['preu_unitari']).replace(',', '.')
    return row

In [0]:
# Funció per crear una nova columna 'unitat' i treure el text "kg" de 'preu_unitari'.
def crear_columna_unitats(df):
    for index, llista_productes in df['products'].items():
        for producte in llista_productes:
            if 'kg' in producte['preu_unitari']:
                producte['quantitat'] = re.sub(r' ?kg ?', '', producte['quantitat'])
                producte['unitat'] = 'kg'
            else:
                producte['unitat'] = 'ud'
    return df

#### Llegim les dades del Data Lake d'Azure

In [0]:
# Utilitzem la funció anterior, per llegir tots els json, i posar-lo a un DF.
import pandas as pd

docs = []

files = list(deep_ls(path)) 

# Recorrem la llista de fitxers.
for file in files: 
    # Llegim els fitxers.
    df = spark.read.json(file.path, multiLine=True)
    
    # Guardem la futa del fitxer
    ruta = str(file.path)

    # Passem el df a df de pandas
    df = df.toPandas()

    # Afegim al df, els camps id_client, id_tiquet, data_tiquet. Ja que aquets camps ens serviarn per classificar els tiquets.
    df['id_tiquet'] = ruta.split("/")[-1].split(".")[0]; # Agragem l'id del tiquet al df que serà el nom del fitxer.
    df['id_client'] = ruta.split("/")[-3] # Agafem l'id del client.
    df['data_scanner_tiquet'] = ruta.split("/")[-2] # Agafem la data.

    # Anem afegint els df a la llista.
    docs.append(df)

# Fem un bucle dintre un array per posar tots els df dintre. Els df que hi ha a la llista docs.
df_docs = pd.concat([df for df in docs], ignore_index=True)


#### Tractament de dades

In [0]:
# Ens assegurem que a la columna 'id_tiquet' no es repeteixin.

# Amb .ducpliacated de pandas, podem agafar els valors duplicats. En aquest cas indiquem del camp id_tiquet
duplicats = df_docs.duplicated(subset=['id_tiquet'])

# Sumem el número de resultats.
num_duplicats = duplicats.sum()

# Eliminem els duplicats i només mentenim la primera aparició
if num_duplicats > 0:
    df_docs = df_docs.drop_duplicates(subset=['id_tiquet'])


In [0]:
# Fem ús de la funció deleteColumns() per eliminar les columnes inecessaries.
columns_to_drop = ['ivas', 'targeta_bancaria', 'NC', 'AUT', 'ARC', 'AID', 'telf', 'nom', 'total_sense_IVA', 'adressa', 'targeta_1', 'targeta_2', 'targeta_bancaria', 'logo_top', 'op', 'logo_contact_less', 'logo_bottom', 'iva_total', 'barcode', 'pagat_targeta_bancaria']

df_docs = deleteColumns(df_docs, columns_to_drop)

In [0]:
# Natajem el camp ciutat treien el codi postal
df_docs['ciutat'] = df_docs['ciutat'].apply(lambda x: x.split(' ')[0])

In [0]:
#  Convertim la columna 'data_scanner_tiquet' a datetime.
df_docs['data_scanner_tiquet'] = pd.to_datetime(df_docs['data_scanner_tiquet'], format='%d%m%Y')

# Ara utilitzem dt.strftime per canviar-li el format.
df_docs['data_scanner_tiquet'] = df_docs['data_scanner_tiquet'].dt.strftime('%d-%m-%Y')

In [0]:
# Creem la nova columna "unitat".
df_docs = crear_columna_unitats(df_docs)

In [0]:
# Canviem els "." per "," a variables de productes.
df_docs = df_docs.apply(canviar_punts_per_comes, axis=1)

In [0]:
# Treiem el text kg de la columna preu_unitari
df_docs = eliminar_txt_kg(df_docs)


#### Emmagatzemem les dades a la capa Silver.


In [0]:
# Penjem les dades particionades amb dies i clients.

# DIES (DATA)
path_dies_silver = 'abfs://{0}@{1}.dfs.core.windows.net/dies/'.format('silver', storage_account)

# CLIENTS
path_clientes_silver = 'abfs://{0}@{1}.dfs.core.windows.net/clients/'.format('silver', storage_account)

# Executem la funció per dies
particionar_dies(df_docs, path_dies_silver)

# Executem la funció per clients
particionar_clients(df_docs, path_clientes_silver)